In [5]:
import tensorflow as tf
import numpy as np
tf.config.run_functions_eagerly(True)

from component.model import get_covid_classifier
from tensorflow import keras
from sklearn.utils import class_weight
from component.vocal_cords import get_vocal_cords_model
from utils.feature_extraction import  get_MFCCS

In [2]:
clf = get_covid_classifier(input_shape=(300, 200, 1),
                 # vocal_cords_path='pretrained_weight/vocal_cords_v1.h5',
                 vocal_cords_path=None,
                 # sentiment_path='pretrained_weight/sentiment_v1.h5',
                 sentiment_path=None,
                 tract_path=None)
clf.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 200, 1) 0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 300, 200, 1)  2           input_1[0][0]                    
__________________________________________________________________________________________________
distribution_lambda (Distributi multiple             0           dense[0][0]                      
__________________________________________________________________________________________________
model_4 (Functional)            (None, 10, 7, 2048)  23581440    distribution_lambda[0][0]        
____________________________________________________________________________________________

In [3]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indexes = set(np.arange(len(X)))

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = random.sample(self.indexes, self.batch_size)
        X = []
        y = []
        for index in indexes:
            try:
                X.append(get_MFCCS(self.X[index]))
            except:
                continue
            y.append(self.y[index])
        return np.array(X), np.array(y, dtype=int)

In [3]:
import glob
X1 = glob.glob("dataset/train/p/*.wav")
X0 = list(glob.glob("dataset/train/n/*.wav"))[:1000]
X = X1 + X0
y = [1]*len(X1) + [0] * len(X0)
print(f"Found {len(X1)} labels positive")
print(f"Found {len(X0)} labels negative")

Found 909 labels positive
Found 1000 labels negative


In [1]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y),
                                                 y)
class_weights = {1 - i: x for i,x in enumerate(class_weights)}
print(f"Class weights: {class_weights}")

NameError: name 'X' is not defined

In [6]:
train_generator = DataGenerator(X_train, y_train, 16)
val_generator = DataGenerator(X_val, y_val, 16)

In [7]:
clf.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['acc'])
history = clf.fit(train_generator, steps_per_epoch=20, validation_steps=20, epochs=50, validation_data=val_generator, class_weight=class_weights,
                  callbacks=[tf.keras.callbacks.ModelCheckpoint('pretrained_weight/model_v1.h5', monitor="val_acc", save_weights_only=True, verbose=1, mode='max', save_best_only=True),
                             tf.keras.callbacks.TensorBoard(log_dir='logs')])
          # callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)])

d:\project\papyrus\venv\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/50
20/20 [==============================] - 23s 883ms/step - loss: 0.6898 - acc: 0.5250 - val_loss: 0.6945 - val_acc: 0.4874

In [6]:
from sklearn.metrics import classification_report
import tqdm
X1_test = glob.glob("split_wavs/testing/p/*.wav")
X0_test = glob.glob("split_wavs/testing/n/*.wav")
X_test = []
y_test = []
for audio_path in tqdm.tqdm(X1_test):
    try:
        X_test.append(get_MFCCS(audio_path))
        y_test.append(1)
    except:
        continue

for audio_path in tqdm.tqdm(X0_test):
    try:
        X_test.append(get_MFCCS(audio_path))
        y_test.append(0)
    except:
        continue
X_test = np.array(X_test)

100%|██████████| 1495/1495 [00:17<00:00, 84.73it/s]


In [11]:
clf.load_weights("pretrained_weight/model_v1.h5")
y_pred = clf.predict(X_test)
y_pred_r = np.rint(y_pred)
print(classification_report(y_pred_r, y_test))

d:\project\papyrus\venv\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


              precision    recall  f1-score   support

         0.0       1.00      0.88      0.94      1249
         1.0       0.00      0.00      0.00         0

    accuracy                           0.88      1249
   macro avg       0.50      0.44      0.47      1249
weighted avg       1.00      0.88      0.94      1249



d:\project\papyrus\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\project\papyrus\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\project\papyrus\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
y_pred


array([[0.4526263],
       [0.4526263],
       [0.4526263],
       ...,
       [0.4526263],
       [0.4526263],
       [0.4526263]], dtype=float32)

In [8]:
from component.model import get_respiratory_tract_model
model = get_respiratory_tract_model(input_shape=(300, 200, 1))
model.load_weights("bio4.h5")
y_pred = model.predict(X_test)
y_pred_r = np.rint(y_pred)
print(classification_report(y_pred_r, y_test))


d:\project\papyrus\venv\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


              precision    recall  f1-score   support

         0.0       0.01      0.62      0.01        16
         1.0       0.97      0.13      0.23      1637

    accuracy                           0.13      1653
   macro avg       0.49      0.38      0.12      1653
weighted avg       0.96      0.13      0.23      1653



In [4]:
import librosa
y, sr = librosa.load(librosa.util.example_audio_file())
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=39)
mfcc_delta = librosa.feature.delta(mfcc, order=1)
mfcc_delta2 = librosa.feature.delta(mfcc, order=2)


